In [19]:
import numpy as np
import cv2
import onnx
from onnx_tf.backend import prepare
import tensorflow as tf

In [3]:
# load onnx model
onnx_path = "model/rps_cuda.onnx"
onnx_model = onnx.load(onnx_path)

In [6]:
# convert .onnx to .pb and export the model
tf_rep = prepare(onnx_model)
tf_rep.export_graph("model/rps_cuda.pb")

INFO:tensorflow:Assets written to: model/rps_cuda.pb/assets


INFO:tensorflow:Assets written to: model/rps_cuda.pb/assets


In [14]:
# convert .pb to .tflite
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model("model/rps_cuda.pb")
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
tflite_model = converter.convert()
open('model/rps_cuda.tflite', 'wb').write(tflite_model)

Estimated count of arithmetic ops: 153.350 M  ops, equivalently 76.675 M  MACs


2022-07-15 10:51:07.849891: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-07-15 10:51:07.849914: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-07-15 10:51:07.850389: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: model/rps_cuda.pb
2022-07-15 10:51:07.854490: I tensorflow/cc/saved_model/reader.cc:81] Reading meta graph with tags { serve }
2022-07-15 10:51:07.854505: I tensorflow/cc/saved_model/reader.cc:122] Reading SavedModel debug info (if present) from: model/rps_cuda.pb
2022-07-15 10:51:07.866870: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2022-07-15 10:51:07.868869: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-07-15 10:51:07.950976: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: model/rps_cuda.pb
2022

12570864

In [15]:
# .tflite test
interpreter = tf.lite.Interpreter(model_path="model/rps_cuda.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape']
input_data = cv2.imread("/home/noah/storage1/datasets/rps_test/rock/noah/220627104025452787.jpg")
input_data = cv2.resize(input_data, (256, 256))
input_data = input_data.astype(np.float32)  # / 127.5 - 1
input_data = np.transpose(input_data, (2, 0, 1))[np.newaxis, ...]
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


[{'name': 'serving_default_input:0',
  'index': 0,
  'shape': array([  1,   3, 256, 256], dtype=int32),
  'shape_signature': array([  1,   3, 256, 256], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

[[9.9999517e-01 3.7316372e-11 3.6784562e-13]]
